In [ ]:
path = "../scraped_data/stateacts"
import os
files = os.listdir(path)

In [92]:
files.pop(0)

'punjab-act.json'

In [93]:
import json
with open(path + "/" + files[0], 'r', encoding='utf-8') as f:
    data = json.load(f)
data

IndexError: list index out of range

In [50]:
def change_discrepancy(actual_year, stated_year,data):
    if data.get(actual_year) is None:
        data[actual_year] = []
        data["document_count"][actual_year] = 0
    data[actual_year].extend(data[stated_year])
    del data[stated_year]
    data["document_count"][actual_year]+=1
    del data["document_count"][stated_year]

    
def save_data(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [85]:
change_discrepancy("1991","2991",data)
save_data(path + "/" + files[0], data)


## Checking for missing data

In [7]:
import json
from time import sleep
from bs4 import BeautifulSoup
import requests

In [8]:
path = "../scraped_data/stateacts"
import os
files = os.listdir(path)

In [9]:
files

['andhra-act.json',
 'arunachal-act.json',
 'assam-act.json',
 'bihar-act.json',
 'chandigarh-act.json',
 'chattisgarh-act.json',
 'delhi-act.json',
 'goa-act.json',
 'gujarat-act.json',
 'haryana-act.json',
 'himachal-act.json',
 'jharkhand-act.json',
 'jk-act.json',
 'karnataka-act.json',
 'madhyabharat-act.json',
 'manipur-act.json',
 'meghalaya-act.json',
 'mh-act.json',
 'mizoram-act.json',
 'mp-act.json',
 'nagaland-act.json',
 'odisha-act.json',
 'puducherry-act.json',
 'punjab-act.json']

In [10]:
def check_missing_data(filename):
    with open(path+"/"+filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    missing_data = []
    for year in list(data.keys())[:-1]:
        for doc in data[year]:
            if doc["data"]=="":
                print(f"Missing data in file: {filename}, year: {year}, doc id: {doc['doc_id']}")
                missing_data.append((filename, year, doc['doc_id']))
    return missing_data



In [23]:
missing_data = []

for file in files:
    w = check_missing_data(file)
    if w:  # only add if it's not None or empty
        missing_data.extend(w)  # extend because w can be a list of tuples


Missing data in file: assam-act.json, year: 2022, doc id: 6b7d93e6-7796-4bf5-beb7-8920c552df8b
Missing data in file: odisha-act.json, year: 1956, doc id: 82c1efb3-77c8-4827-a90f-771cf10350d6
Missing data in file: puducherry-act.json, year: 1970, doc id: d5d6e5c1-f78a-4621-9df9-74d56ed722f7


In [12]:
BASE_URL = "https://indiankanoon.org"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0 Safari/537.36"
    )
}
def attach_missing_data(link):
    print(f"Fetching data for link: {link}")

    data, success = "", False
    for attempt in range(3):  # try max 3 times
        try:
            response = requests.get(link, headers=HEADERS, timeout=10)
            response.raise_for_status()
            sleep(2)  # polite delay

            soup = BeautifulSoup(response.text, "html.parser")
            container = soup.find("div", class_="akn-akomaNtoso")
            if container:
                data = container.get_text(" ", strip=True)
                success = True
                break
        except Exception as e:
            print(f"Attempt {attempt+1} -> {e}")
            sleep(3)

    if not success:
        print(f"❌ Could not fetch data ")
    
    return data


In [ ]:
def add_missing_data(filename, year, doc_id):
    with open(path+"/"+filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for doc in data[year]:
        if doc["doc_id"] == doc_id:
            link = doc["link"]
            temp_data =  attach_missing_data(link)
            doc["data"] = temp_data
            
            
    with open(path+"/"+filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

        

In [ ]:
link = "https://indiankanoon.org/doc/42992215/"
response = requests.get(link,headers=HEADERS)
soup = BeautifulSoup(response.content,"html.parser")
soup.find("div",class_="")

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN"
"http://www.w3.org/TR/html4/strict.dtd">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<link href="/static/js/jquery-ui-1.13.2/jquery-ui.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/js/jquery-ui-1.13.2/jquery-ui.theme.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/search_desktop_v20.css" rel="stylesheet" type="text/css"/>
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-HML2J37TKY"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
            gtag('js', new Date());
            gtag('config', 'G-HML2J37TKY');
    </script>
<title>The Orissa Boiler Attendants' Rules, 1956</title>
</head>
<body>
<div id="tophdr">
<button id="sharelink" type="submit">Share Link</button>
<form action="/change_device/?device=mobile&amp;nextpage=/doc/42992215/" id="we

In [26]:
add_missing_data(missing_data[2][0],missing_data[2][1],missing_data[2][2])

Fetching data for link: https://indiankanoon.org/doc/198591931/
Attempt 1 -> 429 Client Error: Too Many Requests for url: https://indiankanoon.org/doc/198591931/
